### Importing

In [1]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

air_canada = pd.read_csv('../datasets/new_branded_data.csv')

# seperate dataset into training and predicting
air_canada_to_predict = air_canada[air_canada['choice'] != air_canada['choice']]
air_canada = air_canada[air_canada['choice'] == air_canada['choice']]

X, y = air_canada.drop(['Unnamed: 0', 'id', 'ticket_id', 'choice'], axis=1), air_canada[['choice']]

y['choice'] = y['choice'].map({ 'nochoice': 0, 'pref': 1, 'advs': 2 })

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

/tmp/ipykernel_300444/1501615036.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['choice'] = y['choice'].map({ 'nochoice': 0, 'pref': 1, 'advs': 2 })


In [2]:
X.columns

Index(['od', 'flight_departure_datetime', 'purchase_datetime', 'time_diff',
       'trip_type', 'branded_fare', 'number_of_pax', 'ADVS_price',
       'PREF_price', 'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory',
       'PREF_inventory', 'pref_inv_full', 'advs_ratio', 'pref_ratio',
       'total_cap'],
      dtype='object')

### Preprocessing

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [5]:
num_pipeline = make_pipeline(
  StandardScaler()
)

cat_pipeline = make_pipeline(
  OneHotEncoder()
)
preprocessing = ColumnTransformer([
    ('cat', cat_pipeline, ['od', 'trip_type', 'branded_fare', 'total_cap']),
    ('num', num_pipeline, ['time_diff', 'number_of_pax', 'ADVS_price', 'PREF_price', 
                           'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory', 'PREF_inventory',
                           'pref_inv_full', 'advs_ratio', 'pref_ratio'])
  ], remainder='drop')

In [6]:
X_train_prep = preprocessing.fit_transform(X_train)
X_test_prep = preprocessing.transform(X_test)

In [7]:
X_train_prep[:1]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        , -0.60670851,
         0.28654085,  0.58159208, -0.28605495,  0.62180611,  0.58853408,
         0.33954015, -1.19044634,  0.36953924, -1.05137821, -1.3070075 ]])

### Training

In [9]:
from xgboost import XGBClassifier

bst = XGBClassifier(learning_rate=.3, objective='multi:softmax')
bst.fit(X_train_prep, y_train)

y_pred = bst.predict(X_test_prep)

print("Accuracy:", accuracy_score(y_pred, y_test))
print("F1 Score:", f1_score(y_pred, y_test, average='weighted'))

Accuracy: 0.7436363636363637
F1 Score: 0.7801606531476153


### Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],           # Number of boosting rounds
    'learning_rate': [0.01, 0.1, 0.2],         # Step size shrinkage
    'max_depth': [3, 5, 7],                    # Maximum depth of the trees
    'min_child_weight': [1, 3, 5],             # Minimum sum of instance weight needed in a child
    'subsample': [0.6, 0.8, 1.0],              # Subsample ratio of the training instances
    'colsample_bytree': [0.6, 0.8, 1.0],       # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2],                    # Minimum loss reduction to make a further partition on a leaf node
}

rnd_search = GridSearchCV(bst,
    param_grid=param_grid, verbose=3, cv=3, n_jobs=-1,
    scoring='neg_root_mean_squared_error')

rnd_search.fit(X_train_prep, y_train)

In [ ]:
rnd_search.best_params_

In [ ]:
y_pred_best = rnd_search.best_estimator_.predict(X_test_prep)

f1_score(y_pred_best, y_test)

### Predicting

In [10]:
X_to_predict, y_to_predict = air_canada_to_predict.drop(['id', 'ticket_id', 'choice'], axis=1), air_canada_to_predict[['id']]
X_to_predict_prep = preprocessing.transform(X_to_predict)
y_to_predict['choice'] = rnd_search.best_estimator_.predict(X_to_predict_prep)

/tmp/ipykernel_300444/1900845532.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_to_predict['choice'] = bst.predict(X_to_predict_prep)


In [11]:
y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})

/tmp/ipykernel_300444/1816271922.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_to_predict['choice'] = y_to_predict['choice'].map({ 0: 'nochoice', 1: 'pref', 2: 'advs'})


In [12]:
y_final = pd.concat([y_to_predict, air_canada])[['id', 'choice']]

### Saving

In [13]:
pd.DataFrame.to_csv(y_final, '../predictions/pred8.csv', index=False)